<a href="https://colab.research.google.com/github/EnzoLuidge/MIPS/blob/main/Aula10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Sep 17 23:17:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8              14W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Writing exemplo1.cu


In [3]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [4]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


In [5]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Writing exemplo2.cu


In [6]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [7]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


In [8]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Writing stocks.cu


In [9]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [14]:
!./stocks < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00256673
Tempo de CÓPIA (em segundos)  0.134955


In [15]:
%%writefile atv10.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main(){

  int n = 3112;                   // Define o tamanho do vetor (2518 elementos)
  double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
  std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
  auto leitura_i = std::chrono::steady_clock::now();
  thrust::host_vector<double> host(n, 0); // Cria um vetor na CPU com o tamanho especificado

  for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
  }

  auto leitura_f = std::chrono::steady_clock::now(); // Marca o fim da medição do tempo de leitura dos dados
  diff = leitura_f - leitura_i;

  cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

  auto copia_i = std::chrono::steady_clock::now();

  thrust::device_vector<double> dev(host); // Cria um vetor na GPU copiando os dados do vetor host da CPU

  auto copia_f = std::chrono::steady_clock::now();
  diff = copia_f - copia_i;

  cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

  return 0;
}

Overwriting atv10.cu


In [16]:
!nvcc -arch=sm_70 -std=c++14 atv10.cu -o atv10

In [17]:
!./atv10 < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00381688
Tempo de CÓPIA (em segundos)  0.109517


In [33]:
  %%writefile atv10.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/reduce.h>
#include <thrust/extrema.h>
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main(){

  int n = 3112;                   // Define o tamanho do vetor (2518 elementos)
  double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
  std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
  auto leitura_i = std::chrono::steady_clock::now();
  thrust::host_vector<double> host(n, 0); // Cria um vetor na CPU com o tamanho especificado

  for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
  }

  auto leitura_f = std::chrono::steady_clock::now(); // Marca o fim da medição do tempo de leitura dos dados
  diff = leitura_f - leitura_i;

  cout << "Tempo de LEITURA (em segundos)  " << diff.count() << "\n" << endl;

  auto copia_i = std::chrono::steady_clock::now();

  thrust::device_vector<double> dev(host); // Cria um vetor na GPU copiando os dados do vetor host da CPU

  auto copia_f = std::chrono::steady_clock::now();
  diff = copia_f - copia_i;

  cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << "\n" << endl;

  auto calcula_media_i = std::chrono::steady_clock::now();

  double soma = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
  double media = soma / n;

  auto calcula_media_f = std::chrono::steady_clock::now();
  diff = copia_f - copia_i;

  cout << "Média: " << media << endl;
  cout << "Tempo do Calculo da Média (em segundos)  " << diff.count() << "\n" << endl;

  auto calcula_media365_i = std::chrono::steady_clock::now();
  double soma365 = thrust::reduce(dev.end() - 365, dev.end(), 0.0, thrust::plus<double>());
  double media365 = soma365 / 365;

  auto calcula_media365_f = std::chrono::steady_clock::now();
  diff = copia_f - copia_i;
  cout << "Média dos últimos 365 dias: " << media365 << endl;
  cout << "Tempo do Calculo da Média dos últimos 365 dias (em segundos)  " << diff.count() << "\n" << endl;


  auto descobre_maior_menor_total_i = std::chrono::steady_clock::now();

  double maior_total = *thrust::max_element(dev.begin(), dev.end());
  double menor_total = *thrust::min_element(dev.begin(), dev.end());

  auto descobre_maior_menor_total_f = std::chrono::steady_clock::now();

  diff = descobre_maior_menor_total_f - descobre_maior_menor_total_i;

  cout << "Maior valor: " << maior_total << endl;
  cout << "Menor valor: " << menor_total << endl;
  cout << "Tempo do descobre maior e menor valor (em segundos)  " << diff.count() << "\n" << endl;


  auto descobre_maior_menor_365_i = std::chrono::steady_clock::now();

  double maior365 = *thrust::max_element(dev.end() - 365, dev.end());
  double menor365 = *thrust::min_element(dev.end() - 365, dev.end());

  auto descobre_maior_menor_365_f = std::chrono::steady_clock::now();

  diff = descobre_maior_menor_365_f - descobre_maior_menor_365_i;

  cout << "Maior valor dos últimos 365 dias: " << maior365 << endl;
  cout << "Menor valor dos últimos 365 dias: " << menor365 << endl;
  cout << "Tempo do descobre maior e menor valor dos últimos 365 dias (em segundos)  " << diff.count() << endl;

  return 0;
}

Overwriting atv10.cu


In [34]:
!nvcc -arch=sm_70 -std=c++14 atv10.cu -o atv10

In [35]:
!./atv10 < stocks-google.txt

Tempo de LEITURA (em segundos)  0.0028071

Tempo de CÓPIA (em segundos)  0.11427

Média: 1580.08
Tempo do Calculo da Média (em segundos)  0.11427

Média dos últimos 365 dias: 1615.81
Tempo do Calculo da Média dos últimos 365 dias (em segundos)  0.11427

Maior valor: 2200
Menor valor: 1015.24
Tempo do descobre maior e menor valor (em segundos)  0.000482374

Maior valor dos últimos 365 dias: 2200
Menor valor dos últimos 365 dias: 1060.5
Tempo do descobre maior e menor valor dos últimos 365 dias (em segundos)  0.000274203


In [30]:
%%writefile stocks2.txt
150.24,200.34
151.00,199.75
149.75,201.25
152.50,202.00
153.00,203.50
154.25,204.75
155.00,205.50
156.75,206.00
157.50,207.50
158.00,208.25
159.25,209.50
160.00,210.75
161.25,211.00
162.50,212.25
163.00,213.50
164.25,214.75
165.00,215.50
166.50,216.75
167.75,218.00
168.25,219.25
169.00,220.50
170.50,221.75
171.25,223.00
172.50,224.25
173.75,225.50
174.00,226.75
175.25,227.50
176.00,228.25
177.50,229.00
178.00,230.50
179.25,231.75
180.00,233.00
181.50,234.25
182.00,235.50
183.25,236.75
184.50,238.00
185.00,239.25
186.50,240.50
187.75,241.75
188.25,243.00
189.00,244.50
190.50,245.75
191.25,247.00
192.75,248.25
193.00,249.50
194.25,250.75
195.50,252.00
196.00,253.50
197.25,254.75
198.50,256.00
199.00,257.25
200.50,258.50
201.75,259.75
202.25,261.00
203.00,262.25
204.25,263.50
205.50,264.75
206.00,266.00
207.25,267.25
208.50,268.50
209.75,269.75
210.25,271.00
211.50,272.25
212.75,273.50
213.00,274.75
214.25,276.00
215.50,277.25
216.00,278.50
217.50,279.75
218.75,281.00
219.25,282.25
220.50,283.50
221.75,284.75
222.00,286.00
223.50,287.25
224.00,288.50
225.25,289.75
226.50,291.00
227.75,292.25
228.00,293.50
229.50,294.75
230.75,296.00
231.25,297.25
232.50,298.50
233.75,299.75
234.00,301.00
235.50,302.25
236.25,303.50
237.50,304.75
238.75,306.00
239.00,307.25
240.50,308.50
241.75,309.75
242.25,311.00
243.50,312.25
244.75,313.50
245.00,314.75
246.50,316.00
247.25,317.25
248.50,318.50
249.75,319.75
250.00,321.00
251.50,322.25
252.25,323.50
253.50,324.75
254.75,326.00
255.00,327.25
256.50,328.50
257.75,329.75
258.25,331.00
259.50,332.25
260.75,333.50
261.00,334.75
262.50,336.00
263.25,337.25
264.50,338.50
265.75,339.75
266.00,341.00
267.50,342.25
268.25,343.50
269.50,344.75
270.75,346.00
271.00,347.25
272.50,348.50
273.75,349.75
274.25,351.00
275.50,352.25
276.75,353.50
277.00,354.75
278.50,356.00
279.25,357.25
280.50,358.50
281.75,359.75
282.00,361.00
283.50,362.25
284.25,363.50
285.50,364.75
286.75,366.00
287.00,367.25
288.50,368.50
289.75,369.75
290.25,371.00
291.50,372.25
292.75,373.50
293.00,374.75
294.50,376.00
295.25,377.25
296.50,378.50
297.75,379.75
298.00,381.00
299.50,382.25
300.25,383.50
301.50,384.75
302.75,386.00
303.00,387.25
304.50,388.50
305.75,389.75
306.25,391.00
307.50,392.25
308.75,393.50
309.00,394.75
310.50,396.00
311.25,397.25
312.50,398.50
313.75,399.75
314.00,401.00
315.50,402.25
316.25,403.50
317.50,404.75
318.75,406.00
319.00,407.25
320.50,408.50
321.75,409.75
322.25,411.00
323.50,412.25
324.75,413.50
325.00,414.75
326.50,416.00
327.25,417.25
328.50,418.50
329.75,419.75
330.00,421.00
331.50,422.25
332.25,423.50
333.50,424.75
334.75,426.00
335.00,427.25
336.50,428.50
337.75,429.75
338.25,431.00
339.50,432.25
340.75,433.50
341.00,434.75
342.50,436.00
343.25,437.25
344.50,438.50
345.75,439.75
346.00,441.00
347.50,442.25
348.25,443.50
349.50,444.75
350.75,446.00
351.00,447.25
352.50,448.50
353.75,449.75
354.25,451.00
355.50,452.25
356.75,453.50
357.00,454.75
358.50,456.00
359.25,457.25
360.50,458.50
361.75,459.75
362.00,461.00
363.50,462.25
364.25,463.50
365.50,464.75
366.75,466.00
367.00,467.25
368.50,468.50
369.75,469.75
370.25,471.00
371.50,472.25
372.75,473.50
373.00,474.75
374.50,476.00
375.25,477.25
376.50,478.50
377.75,479.75
378.00,481.00
379.50,482.25
380.25,483.50
381.50,484.75
382.75,486.00
383.00,487.25
384.50,488.50
385.75,489.75
386.25,491.00
387.50,492.25
388.75,493.50
389.00,494.75
390.50,496.00
391.25,497.25
392.50,498.50
393.75,499.75
394.00,501.00
395.50,502.25
396.25,503.50
397.50,504.75
398.75,506.00
399.00,507.25
400.50,508.50
401.75,509.75
402.25,511.00
403.50,512.25
404.75,513.50
405.00,514.75
406.50,516.00
407.25,517.25
408.50,518.50
409.75,519.75
410.00,521.00
411.50,522.25
412.25,523.50
413.50,524.75
414.75,526.00
415.00,527.25
416.50,528.50
417.75,529.75
418.25,531.00
419.50,532.25
420.75,533.50
421.00,534.75
422.50,536.00
423.25,537.25
424.50,538.50
425.75,539.75
426.00,541.00
427.50,542.25
428.25,543.50
429.50,544.75
430.75,546.00
431.00,547.25
432.50,548.50
433.75,549.75
434.25,551.00
435.50,552.25
436.75,553.50
437.00,554.75
438.50,556.00
439.25,557.25
440.50,558.50
441.75,559.75
442.00,561.00
443.50,562.25
444.25,563.50
445.50,564.75
446.75,566.00
447.00,567.25
448.50,568.50
449.75,569.75
450.25,571.00
451.50,572.25
452.75,573.50
453.00,574.75
454.50,576.00
455.25,577.25
456.50,578.50
457.75,579.75
458.00,581.00
459.50,582.25
460.25,583.50
461.50,584.75
462.75,586.00
463.00,587.25
464.50,588.50
465.75,589.75
466.25,591.00
467.50,592.25
468.75,593.50
469.00,594.75
470.50,596.00
471.25,597.25
472.50,598.50
473.75,599.75
474.00,601.00
475.50,602.25
476.25,603.50
477.50,604.75
478.75,606.00
479.00,607.25
480.50,608.50
481.75,609.75
482.25,611.00
483.50,612.25
484.75,613.50
485.00,614.75
486.50,616.00
487.25,617.25
488.50,618.50
489.75,619.75
490.00,621.00
491.50,622.25
492.25,623.50
493.50,624.75
494.75,626.00
495.00,627.25
496.50,628.50
497.75,629.75
498.25,631.00
499.50,632.25
500.75,633.50
501.00,634.75
502.50,636.00
503.25,637.25
504.50,638.50
505.75,639.75
506.00,641.00
507.50,642.25
508.25,643.50
509.50,644.75
510.75,646.00
511.00,647.25
512.50,648.50
513.75,649.75
514.25,651.00
515.50,652.25
516.75,653.50
517.00,654.75
518.50,656.00
519.25,657.25
520.50,658.50
521.75,659.75
522.00,661.00
523.50,662.25
524.25,663.50
525.50,664.75
526.75,666.00
527.00,667.25
528.50,668.50
529.75,669.75
530.25,671.00
531.50,672.25
532.75,673.50
533.00,674.75
534.50,676.00
535.25,677.25
536.50,678.50
537.75,679.75
538.00,681.00
539.50,682.25
540.25,683.50
541.50,684.75
542.75,686.00
543.00,687.25
544.50,688.50
545.75,689.75
546.25,691.00
547.50,692.25
548.75,693.50
549.00,694.75
550.50,696.00
551.25,697.25
552.50,698.50
553.75,699.75
554.00,701.00
555.50,702.25
556.25,703.50
557.50,704.75
558.75,706.00
559.00,707.25
560.50,708.50
561.75,709.75
562.25,711.00
563.50,712.25
564.75,713.50
565.00,714.75
566.50,716.00
567.25,717.25
568.50,718.50
569.75,719.75
570.00,721.00
571.50,722.25
572.25,723.50
573.50,724.75
574.75,726.00
575.00,727.25
576.50,728.50
577.75,729.75
578.25,731.00
579.50,732.25
580.75,733.50
581.00,734.75
582.50,736.00
583.25,737.25
584.50,738.50
585.75,739.75
586.00,741.00
587.50,742.25
588.25,743.50
589.50,744.75
590.75,746.00
591.00,747.25
592.50,748.50
593.75,749.75
594.25,751.00
595.50,752.25
596.75,753.50
597.00,754.75
598.50,756.00
599.25,757.25
600.50,758.50
601.75,759.75
602.00,761.00
603.50,762.25
604.25,763.50
605.50,764.75
606.75,766.00
607.00,767.25
608.50,768.50
609.75,769.75
610.25,771.00
611.50,772.25
612.75,773.50
613.00,774.75
614.50,776.00
615.25,777.25
616.50,778.50
617.75,779.75
618.00,781.00
619.50,782.25
620.25,783.50
621.50,784.75
622.75,786.00
623.00,787.25
624.50,788.50
625.75,789.75
626.25,791.00
627.50,792.25
628.75,793.50
629.00,794.75
630.50,796.00
631.25,797.25
632.50,798.50
633.75,799.75
634.00,801.00
635.50,802.25
636.25,803.50
637.50,804.75
638.75,806.00
639.00,807.25
640.50,808.50
641.75,809.75
642.25,811.00
643.50,812.25
644.75,813.50
645.00,814.75
646.50,816.00
647.25,817.25
648.50,818.50
649.75,819.75
650.00,821.00
651.50,822.25
652.25,823.50
653.50,824.75
654.75,826.00
655.00,827.25
656.50,828.50
657.75,829.75
658.25,831.00
659.50,832.25
660.75,833.50
661.00,834.75
662.50,836.00
663.25,837.25
664.50,838.50
665.75,839.75
666.00,841.00
667.50,842.25
668.25,843.50
669.50,844.75
670.75,846.00
671.00,847.25
672.50,848.50
673.75,849.75
674.25,851.00
675.50,852.25
676.75,853.50
677.00,854.75
678.50,856.00
679.25,857.25
680.50,858.50
681.75,859.75
682.00,861.00
683.50,862.25
684.25,863.50
685.50,864.75
686.75,866.00
687.00,867.25
688.50,868.50
689.75,869.75
690.25,871.00
691.50,872.25
692.75,873.50
693.00,874.75
694.50,876.00
695.25,877.25
696.50,878.50
697.75,879.75
698.00,881.00
699.50,882.25
700.25,883.50
701.50,884.75
702.75,886.00
703.00,887.25
704.50,888.50
705.75,889.75
706.25,891.00
707.50,892.25
708.75,893.50
709.00,894.75
710.50,896.00
711.25,897.25
712.50,898.50
713.75,899.75
714.00,901.00
715.50,902.25
716.25,903.50
717.50,904.75
718.75,906.00
719.00,907.25
720.50,908.50
721.75,909.75
722.25,911.00
723.50,912.25
724.75,913.50
725.00,914.75
726.50,916.00
727.25,917.25
728.50,918.50
729.75,919.75
730.00,921.00
731.50,922.25
732.25,923.50
733.50,924.75
734.75,926.00
735.00,927.25
736.50,928.50
737.75,929.75
738.25,931.00
739.50,932.25
740.75,933.50
741.00,934.75
742.50,936.00
743.25,937.25
744.50,938.50
745.75,939.75
746.00,941.00
747.50,942.25
748.25,943.50
749.50,944.75
750.75,946.00
751.00,947.25
752.50,948.50
753.75,949.75
754.25,951.00
755.50,952.25
756.75,953.50
757.00,954.75
758.50,956.00
759.25,957.25
760.50,958.50
761.75,959.75
762.00,961.00
763.50,962.25
764.25,963.50
765.50,964.75
766.75,966.00
767.00,967.25
768.50,968.50
769.75,969.75
770.25,971.00
771.50,972.25
772.75,973.50
773.00,974.75
774.50,976.00
775.25,977.25
776.50,978.50
777.75,979.75
778.00,981.00
779.50,982.25
780.25,983.50
781.50,984.75
782.75,986.00
783.00,987.25
784.50,988.50
785.75,989.75
786.25,991.00
787.50,992.25
788.75,993.50
789.00,994.75
790.50,996.00
791.25,997.25
792.50,998.50
793.75,999.75
794.00,1001.00
795.50,1002.25
796.25,1003.50
797.50,1004.75
798.75,1006.00
799.00,1007.25
800.50,1008.50
801.75,1009.75
802.25,1011.00
803.50,1012.25
804.75,1013.50
805.00,1014.75
806.50,1016.00
807.25,1017.25
808.50,1018.50
809.75,1019.75
810.00,1021.00
811.50,1022.25
812.25,1023.50
813.50,1024.75
814.75,1026.00
815.00,1027.25
816.50,1028.50
817.75,1029.75
818.25,1031.00
819.50,1032.25
820.75,1033.50
821.00,1034.75
822.50,1036.00
823.25,1037.25
824.50,1038.50
825.75,1039.75
826.00,1041.00
827.50,1042.25
828.25,1043.50
829.50,1044.75
830.75,1046.00
831.00,1047.25
832.50,1048.50
833.75,1049.75
834.25,1051.00
835.50,1052.25
836.75,1053.50
837.00,1054.75
838.50,1056.00
839.25,1057.25
840.50,1058.50
841.75,1059.75
842.00,1061.00
843.50,1062.25
844.25,1063.50
845.50,1064.75
846.75,1066.00
847.00,1067.25
848.50,1068.50
849.75,1069.75
850.25,1071.00
851.50,1072.25
852.75,1073.50
853.00,1074.75
854.50,1076.00
855.25,1077.25
856.50,1078.50
857.75,1079.75
858.00,1081.00
859.50,1082.25
860.25,1083.50
861.50,1084.75
862.75,1086.00
863.00,1087.25
864.50,1088.50
865.75,1089.75
866.25,1091.00
867.50,1092.25
868.75,1093.50
869.00,1094.75
870.50,1096.00
871.25,1097.25
872.50,1098.50
873.75,1099.75
874.00,1101.00
875.50,1102.25
876.25,1103.50
877.50,1104.75
878.75,1106.00
879.00,1107.25
880.50,1108.50
881.75,1109.75
882.25,1111.00
883.50,1112.25
884.75,1113.50
885.00,1114.75
886.50,1116.00
887.25,1117.25
888.50,1118.50
889.75,1119.75
890.00,1121.00
891.50,1122.25
892.25,1123.50
893.50,1124.75
894.75,1126.00
895.00,1127.25
896.50,1128.50
897.75,1129.75
898.25,1131.00
899.50,1132.25
900.75,1133.50
901.00,1134.75
902.50,1136.00
903.25,1137.25
904.50,1138.50
905.75,1139.75
906.00,1141.00
907.50,1142.25
908.25,1143.50
909.50,1144.75
910.75,1146.00
911.00,1147.25
912.50,1148.50
913.75,1149.75
914.25,1151.00
915.50,1152.25
916.75,1153.50
917.00,1154.75
918.50,1156.00
919.25,1157.25
920.50,1158.50
921.75,1159.75
922.00,1161.00
923.50,1162.25
924.25,1163.50
925.50,1164.75
926.75,1166.00
927.00,1167.25
928.50,1168.50
929.75,1169.75
930.25,1171.00
931.50,1172.25
932.75,1173.50
933.00,1174.75
934.50,1176.00
935.25,1177.25
936.50,1178.50
937.75,1179.75
938.00,1181.00
939.50,1182.25
940.25,1183.50
941.50,1184.75
942.75,1186.00
943.00,1187.25
944.50,1188.50
945.75,1189.75
946.25,1191.00
947.50,1192.25
948.75,1193.50
949.00,1194.75
950.50,1196.00
951.25,1197.25
952.50,1198.50
953.75,1199.75
954.00,1201.00
955.50,1202.25
956.25,1203.50
957.50,1204.75
958.75,1206.00
959.00,1207.25
960.50,1208.50
961.75,1209.75
962.25,1211.00
963.50,1212.25
964.75,1213.50
965.00,1214.75
966.50,1216.00
967.25,1217.25
968.50,1218.50
969.75,1219.75
970.00,1221.00
971.50,1222.25
972.25,1223.50
973.50,1224.75
974.75,1226.00
975.00,1227.25
976.50,1228.50
977.75,1229.75
978.25,1231.00
979.50,1232.25
980.75,1233.50
981.00,1234.75
982.50,1236.00
983.25,1237.25
984.50,1238.50
985.75,1239.75
986.00,1241.00
987.50,1242.25
988.25,1243.50
989.50,1244.75
990.75,1246.00
991.00,1247.25
992.50,1248.50
993.75,1249.75
994.25,1251.00
995.50,1252.25
996.75,1253.50
997.00,1254.75
998.50,1256.00
999.25,1257.25
1000.50,1258.50
1001.75,1259.75
1002.00,1261.00
1003.50,1262.25
1004.25,1263.50
1005.50,1264.75
1006.75,1266.00
1007.00,1267.25
1008.50,1268.50
1009.75,1269.75
1010.25,1271.00
1011.50,1272.25
1012.75,1273.50
1013.00,1274.75
1014.50,1276.00
1015.25,1277.25
1016.50,1278.50
1017.75,1279.75
1018.00,1281.00
1019.50,1282.25
1020.25,1283.50
1021.50,1284.75
1022.75,1286.00
1023.00,1287.25
1024.50,1288.50
1025.75,1289.75
1026.25,1291.00
1027.50,1292.25
1028.75,1293.50
1029.00,1294.75
1030.50,1296.00
1031.25,1297.25
1032.50,1298.50
1033.75,1299.75
1034.00,1301.00
1035.50,1302.25
1036.25,1303.50
1037.50,1304.75
1038.75,1306.00
1039.00,1307.25
1040.50,1308.50
1041.75,1309.75
1042.25,1311.00
1043.50,1312.25
1044.75,1313.50
1045.00,1314.75
1046.50,1316.00
1047.25,1317.25
1048.50,1318.50
1049.75,1319.75
1050.00,1321.00
1051.50,1322.25
1052.25,1323.50
1053.50,1324.75
1054.75,1326.00
1055.00,1327.25
1056.50,1328.50
1057.75,1329.75
1058.25,1331.00
1059.50,1332.25
1060.75,1333.50
1061.00,1334.75
1062.50,1336.00
1063.25,1337.25
1064.50,1338.50
1065.75,1339.75
1066.00,1341.00
1067.50,1342.25
1068.25,1343.50
1069.50,1344.75
1070.75,1346.00
1071.00,1347.25
1072.50,1348.50
1073.75,1349.75
1074.25,1351.00
1075.50,1352.25
1076.75,1353.50
1077.00,1354.75
1078.50,1356.00
1079.25,1357.25
1080.50,1358.50
1081.75,1359.75
1082.00,1361.00
1083.50,1362.25
1084.25,1363.50
1085.50,1364.75
1086.75,1366.00
1087.00,1367.25
1088.50,1368.50
1089.75,1369.75
1090.25,1371.00
1091.50,1372.25
1092.75,1373.50
1093.00,1374.75
1094.50,1376.00
1095.25,1377.25
1096.50,1378.50
1097.75,1379.75
1098.00,1381.00
1099.50,1382.25
1100.25,1383.50
1101.50,1384.75
1102.75,1386.00
1103.00,1387.25
1104.50,1388.50
1105.75,1389.75
1106.25,1391.00
1107.50,1392.25
1108.75,1393.50
1109.00,1394.75
1110.50,1396.00
1111.25,1397.25
1112.50,1398.50
1113.75,1399.75
1114.00,1401.00
1115.50,1402.25
1116.25,1403.50
1117.50,1404.75
1118.75,1406.00
1119.00,1407.25
1120.50,1408.50
1121.75,1409.75
1122.25,1411.00
1123.50,1412.25
1124.75,1413.50
1125.00,1414.75
1126.50,1416.00
1127.25,1417.25
1128.50,1418.50
1129.75,1419.75
1130.00,1421.00
1131.50,1422.25
1132.25,1423.50
1133.50,1424.75
1134.75,1426.00
1135.00,1427.25
1136.50,1428.50
1137.75,1429.75
1138.25,1431.00
1139.50,1432.25
1140.75,1433.50
1141.00,1434.75
1142.50,1436.00
1143.25,1437.25
1144.50,1438.50
1145.75,1439.75
1146.00,1441.00
1147.50,1442.25
1148.25,1443.50
1149.50,1444.75
1150.75,1446.00
1151.00,1447.25
1152.50,1448.50
1153.75,1449.75
1154.25,1451.00
1155.50,1452.25
1156.75,1453.50
1157.00,1454.75
1158.50,1456.00
1159.25,1457.25
1160.50,1458.50
1161.75,1459.75
1162.00,1461.00
1163.50,1462.25
1164.25,1463.50
1165.50,1464.75
1166.75,1466.00
1167.00,1467.25
1168.50,1468.50
1169.75,1469.75
1170.25,1471.00
1171.50,1472.25
1172.75,1473.50
1173.00,1474.75
1174.50,1476.00
1175.25,1477.25
1176.50,1478.50
1177.75,1479.75
1178.00,1481.00
1179.50,1482.25
1180.25,1483.50
1181.50,1484.75
1182.75,1486.00
1183.00,1487.25
1184.50,1488.50
1185.75,1489.75
1186.25,1491.00
1187.50,1492.25
1188.75,1493.50
1189.00,1494.75
1190.50,1496.00
1191.25,1497.25
1192.50,1498.50
1193.75,1499.75
1194.00,1501.00
1195.50,1502.25
1196.25,1503.50
1197.50,1504.75
1198.75,1506.00
1199.00,1507.25
1200.50,1508.50
1201.75,1509.75
1202.25,1511.00
1203.50,1512.25
1204.75,1513.50
1205.00,1514.75
1206.50,1516.00
1207.25,1517.25
1208.50,1518.50
1209.75,1519.75
1210.00,1521.00
1211.50,1522.25
1212.25,1523.50
1213.50,1524.75
1214.75,1526.00
1215.00,1527.25
1216.50,1528.50
1217.75,1529.75
1218.25,1531.00
1219.50,1532.25
1220.75,1533.50
1221.00,1534.75
1222.50,1536.00
1223.25,1537.25
1224.50,1538.50
1225.75,1539.75
1226.00,1541.00
1227.50,1542.25
1228.25,1543.50
1229.50,1544.75
1230.75,1546.00
1231.00,1547.25
1232.50,1548.50
1233.75,1549.75
1234.25,1551.00
1235.50,1552.25
1236.75,1553.50
1237.00,1554.75
1238.50,1556.00
1239.25,1557.25
1240.50,1558.50
1241.75,1559.75
1242.00,1561.00
1243.50,1562.25
1244.25,1563.50
1245.50,1564.75
1246.75,1566.00
1247.00,1567.25
1248.50,1568.50
1249.75,1569.75
1250.25,1571.00
1251.50,1572.25
1252.75,1573.50
1253.00,1574.75
1254.50,1576.00
1255.25,1577.25
1256.50,1578.50
1257.75,1579.75
1258.00,1581.00
1259.50,1582.25
1260.25,1583.50
1261.50,1584.75
1262.75,1586.00
1263.00,1587.25
1264.50,1588.50
1265.75,1589.75
1266.25,1591.00
1267.50,1592.25
1268.75,1593.50
1269.00,1594.75
1270.50,1596.00
1271.25,1597.25
1272.50,1598.50
1273.75,1599.75
1274.00,1601.00
1275.50,1602.25
1276.25,1603.50
1277.50,1604.75
1278.75,1606.00
1279.00,1607.25
1280.50,1608.50
1281.75,1609.75
1282.25,1611.00
1283.50,1612.25
1284.75,1613.50
1285.00,1614.75
1286.50,1616.00
1287.25,1617.25
1288.50,1618.50
1289.75,1619.75
1290.00,1621.00
1291.50,1622.25
1292.25,1623.50
1293.50,1624.75
1294.75,1626.00
1295.00,1627.25
1296.50,1628.50
1297.75,1629.75
1298.25,1631.00
1299.50,1632.25
1300.75,1633.50
1301.00,1634.75
1302.50,1636.00
1303.25,1637.25
1304.50,1638.50
1305.75,1639.75
1306.00,1641.00
1307.50,1642.25
1308.25,1643.50
1309.50,1644.75
1310.75,1646.00
1311.00,1647.25
1312.50,1648.50
1313.75,1649.75
1314.25,1651.00
1315.50,1652.25
1316.75,1653.50
1317.00,1654.75
1318.50,1656.00
1319.25,1657.25
1320.50,1658.50
1321.75,1659.75
1322.00,1661.00
1323.50,1662.25
1324.25,1663.50
1325.50,1664.75
1326.75,1666.00
1327.00,1667.25
1328.50,1668.50
1329.75,1669.75
1330.25,1671.00
1331.50,1672.25
1332.75,1673.50
1333.00,1674.75
1334.50,1676.00
1335.25,1677.25
1336.50,1678.50
1337.75,1679.75
1338.00,1681.00
1339.50,1682.25
1340.25,1683.50
1341.50,1684.75
1342.75,1686.00
1343.00,1687.25
1344.50,1688.50
1345.75,1689.75
1346.25,1691.00
1347.50,1692.25
1348.75,1693.50
1349.00,1694.75
1350.50,1696.00
1351.25,1697.25
1352.50,1698.50
1353.75,1699.75
1354.00,1701.00
1355.50,1702.25
1356.25,1703.50
1357.50,1704.75
1358.75,1706.00
1359.00,1707.25
1360.50,1708.50
1361.75,1709.75
1362.25,1711.00
1363.50,1712.25
1364.75,1713.50
1365.00,1714.75
1366.50,1716.00
1367.25,1717.25
1368.50,1718.50
1369.75,1719.75
1370.00,1721.00
1371.50,1722.25
1372.25,1723.50
1373.50,1724.75
1374.75,1726.00
1375.00,1727.25
1376.50,1728.50
1377.75,1729.75
1378.25,1731.00
1379.50,1732.25
1380.75,1733.50
1381.00,1734.75
1382.50,1736.00
1383.25,1737.25
1384.50,1738.50
1385.75,1739.75
1386.00,1741.00
1387.50,1742.25
1388.25,1743.50
1389.50,1744.75
1390.75,1746.00
1391.00,1747.25
1392.50,1748.50
1393.75,1749.75
1394.25,1751.00
1395.50,1752.25
1396.75,1753.50
1397.00,1754.75
1398.50,1756.00
1399.25,1757.25
1400.50,1758.50
1401.75,1759.75
1402.00,1761.00
1403.50,1762.25
1404.25,1763.50
1405.50,1764.75
1406.75,1766.00
1407.00,1767.25
1408.50,1768.50
1409.75,1769.75
1410.25,1771.00
1411.50,1772.25
1412.75,1773.50
1413.00,1774.75
1414.50,1776.00
1415.25,1777.25
1416.50,1778.50
1417.75,1779.75
1418.00,1781.00
1419.50,1782.25
1420.25,1783.50
1421.50,1784.75
1422.75,1786.00
1423.00,1787.25
1424.50,1788.50
1425.75,1789.75
1426.25,1791.00
1427.50,1792.25
1428.75,1793.50
1429.00,1794.75
1430.50,1796.00
1431.25,1797.25
1432.50,1798.50
1433.75,1799.75
1434.00,1801.00
1435.50,1802.25
1436.25,1803.50
1437.50,1804.75
1438.75,1806.00
1439.00,1807.25
1440.50,1808.50
1441.75,1809.75
1442.25,1811.00
1443.50,1812.25
1444.75,1813.50
1445.00,1814.75
1446.50,1816.00
1447.25,1817.25
1448.50,1818.50
1449.75,1819.75
1450.00,1821.00
1451.50,1822.25
1452.25,1823.50
1453.50,1824.75
1454.75,1826.00
1455.00,1827.25
1456.50,1828.50
1457.75,1829.75
1458.25,1831.00
1459.50,1832.25
1460.75,1833.50
1461.00,1834.75
1462.50,1836.00
1463.25,1837.25
1464.50,1838.50
1465.75,1839.75
1466.00,1841.00
1467.50,1842.25
1468.25,1843.50
1469.50,1844.75
1470.75,1846.00
1471.00,1847.25
1472.50,1848.50
1473.75,1849.75
1474.25,1851.00
1475.50,1852.25
1476.75,1853.50
1477.00,1854.75
1478.50,1856.00
1479.25,1857.25
1480.50,1858.50
1481.75,1859.75
1482.00,1861.00
1483.50,1862.25
1484.25,1863.50
1485.50,1864.75
1486.75,1866.00
1487.00,1867.25
1488.50,1868.50
1489.75,1869.75
1490.25,1871.00
1491.50,1872.25
1492.75,1873.50
1493.00,1874.75
1494.50,1876.00
1495.25,1877.25
1496.50,1878.50
1497.75,1879.75
1498.00,1881.00
1499.50,1882.25
1500.25,1883.50
1501.50,1884.75
1502.75,1886.00
1503.00,1887.25
1504.50,1888.50
1505.75,1889.75
1506.25,1891.00
1507.50,1892.25
1508.75,1893.50
1509.00,1894.75
1510.50,1896.00
1511.25,1897.25
1512.50,1898.50
1513.75,1899.75
1514.00,1901.00
1515.50,1902.25
1516.25,1903.50
1517.50,1904.75
1518.75,1906.00
1519.00,1907.25
1520.50,1908.50
1521.75,1909.75
1522.25,1911.00
1523.50,1912.25
1524.75,1913.50
1525.00,1914.75
1526.50,1916.00
1527.25,1917.25
1528.50,1918.50
1529.75,1919.75
1530.00,1921.00
1531.50,1922.25
1532.25,1923.50
1533.50,1924.75
1534.75,1926.00
1535.00,1927.25
1536.50,1928.50
1537.75,1929.75
1538.25,1931.00
1539.50,1932.25
1540.75,1933.50
1541.00,1934.75
1542.50,1936.00
1543.25,1937.25
1544.50,1938.50
1545.75,1939.75
1546.00,1941.00
1547.50,1942.25
1548.25,1943.50
1549.50,1944.75
1550.75,1946.00
1551.00,1947.25
1552.50,1948.50
1553.75,1949.75
1554.25,1951.00
1555.50,1952.25
1556.75,1953.50
1557.00,1954.75
1558.50,1956.00
1559.25,1957.25
1560.50,1958.50
1561.75,1959.75
1562.00,1961.00
1563.50,1962.25
1564.25,1963.50
1565.50,1964.75
1566.75,1966.00
1567.00,1967.25
1568.50,1968.50
1569.75,1969.75
1570.25,1971.00
1571.50,1972.25
1572.75,1973.50
1573.00,1974.75
1574.50,1976.00
1575.25,1977.25
1576.50,1978.50
1577.75,1979.75
1578.00,1981.00
1579.50,1982.25
1580.25,1983.50
1581.50,1984.75
1582.75,1986.00
1583.00,1987.25
1584.50,1988.50
1585.75,1989.75
1586.25,1991.00
1587.50,1992.25
1588.75,1993.50
1589.00,1994.75
1590.50,1996.00
1591.25,1997.25
1592.50,1998.50
1593.75,1999.75
1594.00,2001.00
1595.50,2002.25
1596.25,2003.50
1597.50,2004.75
1598.75,2006.00
1599.00,2007.25
1600.50,2008.50
1601.75,2009.75
1602.25,2011.00
1603.50,2012.25
1604.75,2013.50
1605.00,2014.75
1606.50,2016.00
1607.25,2017.25
1608.50,2018.50
1609.75,2019.75
1610.00,2021.00
1611.50,2022.25
1612.25,2023.50
1613.50,2024.75
1614.75,2026.00
1615.00,2027.25
1616.50,2028.50
1617.75,2029.75
1618.25,2031.00
1619.50,2032.25
1620.75,2033.50
1621.00,2034.75
1622.50,2036.00
1623.25,2037.25
1624.50,2038.50
1625.75,2039.75
1626.00,2041.00
1627.50,2042.25
1628.25,2043.50
1629.50,2044.75
1630.75,2046.00
1631.00,2047.25
1632.50,2048.50
1633.75,2049.75
1634.25,2051.00
1635.50,2052.25
1636.75,2053.50
1637.00,2054.75
1638.50,2056.00
1639.25,2057.25
1640.50,2058.50
1641.75,2059.75
1642.00,2061.00
1643.50,2062.25
1644.25,2063.50
1645.50,2064.75
1646.75,2066.00
1647.00,2067.25
1648.50,2068.50
1649.75,2069.75
1650.25,2071.00
1651.50,2072.25
1652.75,2073.50
1653.00,2074.75
1654.50,2076.00
1655.25,2077.25
1656.50,2078.50
1657.75,2079.75
1658.00,2081.00
1659.50,2082.25
1660.25,2083.50
1661.50,2084.75
1662.75,2086.00
1663.00,2087.25
1664.50,2088.50
1665.75,2089.75
1666.25,2091.00
1667.50,2092.25
1668.75,2093.50
1669.00,2094.75
1670.50,2096.00
1671.25,2097.25
1672.50,2098.50
1673.75,2099.75
1674.00,2101.00
1675.50,2102.25
1676.25,2103.50
1677.50,2104.75
1678.75,2106.00
1679.00,2107.25
1680.50,2108.50
1681.75,2109.75
1682.25,2111.00
1683.50,2112.25
1684.75,2113.50
1685.00,2114.75
1686.50,2116.00
1687.25,2117.25
1688.50,2118.50
1689.75,2119.75
1690.00,2121.00
1691.50,2122.25
1692.25,2123.50
1693.50,2124.75
1694.75,2126.00
1695.00,2127.25
1696.50,2128.50
1697.75,2129.75
1698.25,2131.00
1699.50,2132.25
1700.75,2133.50
1701.00,2134.75
1702.50,2136.00
1703.25,2137.25
1704.50,2138.50
1705.75,2139.75
1706.00,2141.00
1707.50,2142.25
1708.25,2143.50
1709.50,2144.75
1710.75,2146.00
1711.00,2147.25
1712.50,2148.50
1713.75,2149.75
1714.25,2151.00
1715.50,2152.25
1716.75,2153.50
1717.00,2154.75
1718.50,2156.00
1719.25,2157.25
1720.50,2158.50
1721.75,2159.75
1722.00,2161.00
1723.50,2162.25
1724.25,2163.50
1725.50,2164.75
1726.75,2166.00
1727.00,2167.25
1728.50,2168.50
1729.75,2169.75
1730.25,2171.00
1731.50,2172.25
1732.75,2173.50
1733.00,2174.75
1734.50,2176.00
1735.25,2177.25
1736.50,2178.50
1737.75,2179.75
1738.00,2181.00
1739.50,2182.25
1740.25,2183.50
1741.50,2184.75
1742.75,2186.00
1743.00,2187.25
1744.50,2188.50
1745.75,2189.75
1746.25,2191.00
1747.50,2192.25
1748.75,2193.50
1749.00,2194.75
1750.50,2196.00
1751.25,2197.25
1752.50,2198.50
1753.75,2199.75
1754.00,2201.00
1755.50,2202.25
1756.25,2203.50
1757.50,2204.75
1758.75,2206.00
1759.00,2207.25
1760.50,2208.50
1761.75,2209.75
1762.25,2211.00
1763.50,2212.25
1764.75,2213.50
1765.00,2214.75
1766.50,2216.00
1767.25,2217.25
1768.50,2218.50
1769.75,2219.75
1770.00,2221.00
1771.50,2222.25
1772.25,2223.50
1773.50,2224.75
1774.75,2226.00
1775.00,2227.25
1776.50,2228.50
1777.75,2229.75
1778.25,2231.00
1779.50,2232.25
1780.75,2233.50
1781.00,2234.75
1782.50,2236.00
1783.25,2237.25
1784.50,2238.50
1785.75,2239.75
1786.00,2241.00
1787.50,2242.25
1788.25,2243.50
1789.50,2244.75
1790.75,2246.00
1791.00,2247.25
1792.50,2248.50
1793.75,2249.75
1794.25,2251.00
1795.50,2252.25
1796.75,2253.50
1797.00,2254.75
1798.50,2256.00
1799.25,2257.25
1800.50,2258.50
1801.75,2259.75
1802.00,2261.00
1803.50,2262.25
1804.25,2263.50
1805.50,2264.75
1806.75,2266.00
1807.00,2267.25
1808.50,2268.50
1809.75,2269.75
1810.25,2271.00
1811.50,2272.25
1812.75,2273.50
1813.00,2274.75
1814.50,2276.00
1815.25,2277.25
1816.50,2278.50
1817.75,2279.75
1818.00,2281.00
1819.50,2282.25
1820.25,2283.50
1821.50,2284.75
1822.75,2286.00
1823.00,2287.25
1824.50,2288.50
1825.75,2289.75
1826.25,2291.00
1827.50,2292.25
1828.75,2293.50
1829.00,2294.75
1830.50,2296.00
1831.25,2297.25
1832.50,2298.50
1833.75,2299.75
1834.00,2301.00
1835.50,2302.25
1836.25,2303.50
1837.50,2304.75
1838.75,2306.00
1839.00,2307.25
1840.50,2308.50
1841.75,2309.75
1842.25,2311.00
1843.50,2312.25
1844.75,2313.50
1845.00,2314.75
1846.50,2316.00
1847.25,2317.25
1848.50,2318.50
1849.75,2319.75
1850.00,2321.00
1851.50,2322.25
1852.25,2323.50
1853.50,2324.75
1854.75,2326.00
1855.00,2327.25
1856.50,2328.50
1857.75,2329.75
1858.25,2331.00
1859.50,2332.25
1860.75,2333.50
1861.00,2334.75
1862.50,2336.00
1863.25,2337.25
1864.50,2338.50
1865.75,2339.75
1866.00,2341.00
1867.50,2342.25
1868.25,2343.50
1869.50,2344.75
1870.75,2346.00
1871.00,2347.25
1872.50,2348.50
1873.75,2349.75
1874.25,2351.00
1875.50,2352.25
1876.75,2353.50
1877.00,2354.75
1878.50,2356.00
1879.25,2357.25
1880.50,2358.50
1881.75,2359.75
1882.00,2361.00
1883.50,2362.25
1884.25,2363.50
1885.50,2364.75
1886.75,2366.00
1887.00,2367.25
1888.50,2368.50
1889.75,2369.75
1890.25,2371.00
1891.50,2372.25
1892.75,2373.50
1893.00,2374.75
1894.50,2376.00
1895.25,2377.25
1896.50,2378.50
1897.75,2379.75
1898.00,2381.00
1899.50,2382.25
1900.25,2383.50
1901.50,2384.75
1902.75,2386.00
1903.00,2387.25
1904.50,2388.50
1905.75,2389.75
1906.25,2391.00
1907.50,2392.25
1908.75,2393.50
150.24,200.34
151.00,199.75
149.75,201.25
152.50,202.00
153.00,203.50
154.25,204.75
155.00,205.50
156.75,206.00
157.50,207.50
158.00,208.25
159.25,209.50
160.00,210.75
161.25,211.00
162.50,212.25
163.00,213.50
164.25,214.75
165.00,215.50
166.50,216.75
167.75,218.00
168.25,219.25
169.00,220.50
170.50,221.75
171.25,223.00
172.50,224.25
173.75,225.50
174.00,226.75
175.25,227.50
176.00,228.25
177.50,229.00
178.00,230.50
179.25,231.75
180.00,233.00
181.50,234.25
182.00,235.50
183.25,236.75
184.50,238.00
185.00,239.25
186.50,240.50
187.75,241.75
188.25,243.00
189.00,244.50
190.50,245.75
191.25,247.00
192.75,248.25
193.00,249.50
194.25,250.75
195.50,252.00
196.00,253.50
197.25,254.75
198.50,256.00
199.00,257.25
200.50,258.50
201.75,259.75
202.25,261.00
203.00,262.25
204.25,263.50
205.50,264.75
206.00,266.00
207.25,267.25
208.50,268.50
209.75,269.75
210.25,271.00
211.50,272.25
212.75,273.50
213.00,274.75
214.25,276.00
215.50,277.25
216.00,278.50
217.50,279.75
218.75,281.00
219.25,282.25
220.50,283.50
221.75,284.75
222.00,286.00
223.50,287.25
224.00,288.50
225.25,289.75
226.50,291.00
227.75,292.25
228.00,293.50
229.50,294.75
230.75,296.00
231.25,297.25
232.50,298.50
233.75,299.75
234.00,301.00
235.50,302.25
236.25,303.50
237.50,304.75
238.75,306.00
239.00,307.25
240.50,308.50
241.75,309.75
242.25,311.00
243.50,312.25
244.75,313.50
245.00,314.75
246.50,316.00
247.25,317.25
248.50,318.50
249.75,319.75
250.00,321.00
251.50,322.25
252.25,323.50
253.50,324.75
254.75,326.00
255.00,327.25
256.50,328.50
257.75,329.75
258.25,331.00
259.50,332.25
260.75,333.50
261.00,334.75
262.50,336.00
263.25,337.25
264.50,338.50
265.75,339.75
266.00,341.00
267.50,342.25
268.25,343.50
269.50,344.75
270.75,346.00
271.00,347.25
272.50,348.50
273.75,349.75
274.25,351.00
275.50,352.25
276.75,353.50
277.00,354.75
278.50,356.00
279.25,357.25
280.50,358.50
281.75,359.75
282.00,361.00
283.50,362.25
284.25,363.50
285.50,364.75
286.75,366.00
287.00,367.25
288.50,368.50
289.75,369.75
290.25,371.00
291.50,372.25
292.75,373.50
293.00,374.75
294.50,376.00
295.25,377.25
296.50,378.50
297.75,379.75
298.00,381.00
299.50,382.25
300.25,383.50
301.50,384.75
302.75,386.00
303.00,387.25
304.50,388.50
305.75,389.75
306.25,391.00
307.50,392.25
308.75,393.50
309.00,394.75
310.50,396.00
311.25,397.25
312.50,398.50
313.75,399.75
314.00,401.00
315.50,402.25
316.25,403.50
317.50,404.75
318.75,406.00
319.00,407.25
320.50,408.50
321.75,409.75
322.25,411.00
323.50,412.25
324.75,413.50
325.00,414.75
326.50,416.00
327.25,417.25
328.50,418.50
329.75,419.75
330.00,421.00
331.50,422.25
332.25,423.50
333.50,424.75
334.75,426.00
335.00,427.25
336.50,428.50
337.75,429.75
338.25,431.00
339.50,432.25
340.75,433.50
341.00,434.75
342.50,436.00
343.25,437.25
344.50,438.50
345.75,439.75
346.00,441.00
347.50,442.25
348.25,443.50
349.50,444.75
350.75,446.00
351.00,447.25
352.50,448.50
353.75,449.75
354.25,451.00
355.50,452.25
356.75,453.50
357.00,454.75
358.50,456.00
359.25,457.25
360.50,458.50
361.75,459.75
362.00,461.00
363.50,462.25
364.25,463.50
365.50,464.75
366.75,466.00
367.00,467.25
368.50,468.50
369.75,469.75
370.25,471.00
371.50,472.25
372.75,473.50
373.00,474.75
374.50,476.00
375.25,477.25
376.50,478.50
377.75,479.75
378.00,481.00
379.50,482.25
380.25,483.50
381.50,484.75
382.75,486.00
383.00,487.25
384.50,488.50
385.75,489.75
386.25,491.00
387.50,492.25
388.75,493.50
389.00,494.75
390.50,496.00
391.25,497.25
392.50,498.50
393.75,499.75
394.00,501.00
395.50,502.25
396.25,503.50
397.50,504.75
398.75,506.00
399.00,507.25
400.50,508.50
401.75,509.75
402.25,511.00
403.50,512.25
404.75,513.50
405.00,514.75
406.50,516.00
407.25,517.25
408.50,518.50
409.75,519.75
410.00,521.00
411.50,522.25
412.25,523.50
413.50,524.75
414.75,526.00
415.00,527.25
416.50,528.50
417.75,529.75
418.25,531.00
419.50,532.25
420.75,533.50
421.00,534.75
422.50,536.00
423.25,537.25
424.50,538.50
425.75,539.75
426.00,541.00
427.50,542.25
428.25,543.50
429.50,544.75
430.75,546.00
431.00,547.25
432.50,548.50
433.75,549.75
434.25,551.00
435.50,552.25
436.75,553.50
437.00,554.75
438.50,556.00
439.25,557.25
440.50,558.50
441.75,559.75
442.00,561.00
443.50,562.25
444.25,563.50
445.50,564.75
446.75,566.00
447.00,567.25
448.50,568.50
449.75,569.75
450.25,571.00
451.50,572.25
452.75,573.50
453.00,574.75
454.50,576.00
455.25,577.25
456.50,578.50
457.75,579.75
458.00,581.00
459.50,582.25
460.25,583.50
461.50,584.75
462.75,586.00
463.00,587.25
464.50,588.50
465.75,589.75
466.25,591.00
467.50,592.25
468.75,593.50
469.00,594.75
470.50,596.00
471.25,597.25
472.50,598.50
473.75,599.75
474.00,601.00
475.50,602.25
476.25,603.50
477.50,604.75
478.75,606.00
479.00,607.25
480.50,608.50
481.75,609.75
482.25,611.00
483.50,612.25
484.75,613.50
485.00,614.75
486.50,616.00
487.25,617.25
488.50,618.50
489.75,619.75
490.00,621.00
491.50,622.25
492.25,623.50
493.50,624.75
494.75,626.00
495.00,627.25
496.50,628.50
497.75,629.75
498.25,631.00
499.50,632.25
500.75,633.50
501.00,634.75
502.50,636.00
503.25,637.25
504.50,638.50
505.75,639.75
506.00,641.00
507.50,642.25
508.25,643.50
509.50,644.75
510.75,646.00
511.00,647.25
512.50,648.50
513.75,649.75
514.25,651.00
515.50,652.25
516.75,653.50
517.00,654.75
518.50,656.00
519.25,657.25
520.50,658.50
521.75,659.75
522.00,661.00
523.50,662.25
524.25,663.50
525.50,664.75
526.75,666.00
527.00,667.25
528.50,668.50
529.75,669.75
530.25,671.00
531.50,672.25
532.75,673.50
533.00,674.75
534.50,676.00
535.25,677.25
536.50,678.50
537.75,679.75
538.00,681.00
539.50,682.25
540.25,683.50
541.50,684.75
542.75,686.00
543.00,687.25
544.50,688.50
545.75,689.75
546.25,691.00
547.50,692.25
548.75,693.50
549.00,694.75
550.50,696.00
551.25,697.25
552.50,698.50
553.75,699.75
554.00,701.00
555.50,702.25
556.25,703.50
557.50,704.75
558.75,706.00
559.00,707.25
560.50,708.50
561.75,709.75
562.25,711.00
563.50,712.25
564.75,713.50
565.00,714.75
566.50,716.00
567.25,717.25
568.50,718.50
569.75,719.75
570.00,721.00
571.50,722.25
572.25,723.50
573.50,724.75
574.75,726.00
575.00,727.25
576.50,728.50
577.75,729.75
578.25,731.00
579.50,732.25
580.75,733.50
581.00,734.75
582.50,736.00
583.25,737.25
584.50,738.50
585.75,739.75
586.00,741.00
587.50,742.25
588.25,743.50
589.50,744.75
590.75,746.00
591.00,747.25
592.50,748.50
593.75,749.75
594.25,751.00
595.50,752.25
596.75,753.50
597.00,754.75
598.50,756.00
599.25,757.25
600.50,758.50
601.75,759.75
602.00,761.00
603.50,762.25
604.25,763.50
605.50,764.75
606.75,766.00
607.00,767.25
608.50,768.50
609.75,769.75
610.25,771.00
611.50,772.25
612.75,773.50
613.00,774.75
614.50,776.00
615.25,777.25
616.50,778.50
617.75,779.75
618.00,781.00
619.50,782.25
620.25,783.50
621.50,784.75
622.75,786.00
623.00,787.25
624.50,788.50
625.75,789.75
626.25,791.00
627.50,792.25
628.75,793.50
629.00,794.75
630.50,796.00
631.25,797.25
632.50,798.50
633.75,799.75
634.00,801.00
635.50,802.25
636.25,803.50
637.50,804.75
638.75,806.00
639.00,807.25
640.50,808.50
641.75,809.75
642.25,811.00
643.50,812.25
644.75,813.50
645.00,814.75
646.50,816.00
647.25,817.25
648.50,818.50
649.75,819.75
650.00,821.00
651.50,822.25
652.25,823.50
653.50,824.75
654.75,826.00
655.00,827.25
656.50,828.50
657.75,829.75
658.25,831.00
659.50,832.25
660.75,833.50
661.00,834.75
662.50,836.00
663.25,837.25
664.50,838.50
665.75,839.75
666.00,841.00
667.50,842.25
668.25,843.50
669.50,844.75
670.75,846.00
671.00,847.25
672.50,848.50
673.75,849.75
674.25,851.00
675.50,852.25
676.75,853.50
677.00,854.75
678.50,856.00
679.25,857.25
680.50,858.50
681.75,859.75
682.00,861.00
683.50,862.25
684.25,863.50
685.50,864.75
686.75,866.00
687.00,867.25
688.50,868.50
689.75,869.75
690.25,871.00
691.50,872.25
692.75,873.50
693.00,874.75
694.50,876.00
695.25,877.25
696.50,878.50
697.75,879.75
698.00,881.00
699.50,882.25
700.25,883.50
701.50,884.75
702.75,886.00
703.00,887.25
704.50,888.50
705.75,889.75
706.25,891.00
707.50,892.25
708.75,893.50
709.00,894.75
710.50,896.00
711.25,897.25
712.50,898.50
713.75,899.75
714.00,901.00
715.50,902.25
716.25,903.50
717.50,904.75
718.75,906.00
719.00,907.25
720.50,908.50
721.75,909.75
722.25,911.00
723.50,912.25
724.75,913.50
725.00,914.75
726.50,916.00
727.25,917.25
728.50,918.50
729.75,919.75
730.00,921.00
731.50,922.25
732.25,923.50
733.50,924.75
734.75,926.00
735.00,927.25
736.50,928.50
737.75,929.75
738.25,931.00
739.50,932.25
740.75,933.50
741.00,934.75
742.50,936.00
743.25,937.25
744.50,938.50
745.75,939.75
746.00,941.00
747.50,942.25
748.25,943.50
749.50,944.75
750.75,946.00
751.00,947.25
752.50,948.50
753.75,949.75
754.25,951.00
755.50,952.25
756.75,953.50
757.00,954.75
758.50,956.00
759.25,957.25
760.50,958.50
761.75,959.75
762.00,961.00
763.50,962.25
764.25,963.50
765.50,964.75
766.75,966.00
767.00,967.25
768.50,968.50
769.75,969.75
770.25,971.00
771.50,972.25
772.75,973.50
773.00,974.75
774.50,976.00
775.25,977.25
776.50,978.50
777.75,979.75
778.00,981.00
779.50,982.25
780.25,983.50
781.50,984.75
782.75,986.00
783.00,987.25
784.50,988.50
785.75,989.75
786.25,991.00
787.50,992.25
788.75,993.50
789.00,994.75
790.50,996.00
791.25,997.25
792.50,998.50
793.75,999.75
794.00,1001.00
795.50,1002.25
796.25,1003.50
797.50,1004.75
798.75,1006.00
799.00,1007.25
800.50,1008.50
801.75,1009.75
802.25,1011.00
803.50,1012.25
804.75,1013.50
805.00,1014.75
806.50,1016.00
807.25,1017.25
808.50,1018.50
809.75,1019.75
810.00,1021.00
811.50,1022.25
812.25,1023.50
813.50,1024.75
814.75,1026.00
815.00,1027.25
816.50,1028.50
817.75,1029.75
818.25,1031.00
819.50,1032.25
820.75,1033.50
821.00,1034.75
822.50,1036.00
823.25,1037.25
824.50,1038.50
825.75,1039.75
826.00,1041.00
827.50,1042.25
828.25,1043.50
829.50,1044.75
830.75,1046.00
831.00,1047.25
832.50,1048.50
833.75,1049.75
834.25,1051.00
835.50,1052.25
836.75,1053.50
837.00,1054.75
838.50,1056.00
839.25,1057.25
840.50,1058.50
841.75,1059.75
842.00,1061.00
843.50,1062.25
844.25,1063.50
845.50,1064.75
846.75,1066.00
847.00,1067.25
848.50,1068.50
849.75,1069.75
850.25,1071.00
851.50,1072.25
852.75,1073.50
853.00,1074.75
854.50,1076.00
855.25,1077.25
856.50,1078.50
857.75,1079.75
858.00,1081.00
859.50,1082.25
860.25,1083.50
861.50,1084.75
862.75,1086.00
863.00,1087.25
864.50,1088.50
865.75,1089.75
866.25,1091.00
867.50,1092.25
868.75,1093.50
869.00,1094.75
870.50,1096.00
871.25,1097.25
872.50,1098.50
873.75,1099.75
874.00,1101.00
875.50,1102.25
876.25,1103.50
877.50,1104.75
878.75,1106.00
879.00,1107.25
880.50,1108.50
881.75,1109.75
882.25,1111.00
883.50,1112.25
884.75,1113.50
885.00,1114.75
886.50,1116.00
887.25,1117.25
888.50,1118.50
889.75,1119.75
890.00,1121.00
891.50,1122.25
892.25,1123.50
893.50,1124.75
894.75,1126.00
895.00,1127.25
896.50,1128.50
897.75,1129.75
898.25,1131.00
899.50,1132.25
900.75,1133.50
901.00,1134.75
902.50,1136.00
903.25,1137.25
904.50,1138.50
905.75,1139.75
906.00,1141.00
907.50,1142.25
908.25,1143.50
909.50,1144.75
910.75,1146.00
911.00,1147.25
912.50,1148.50
913.75,1149.75
914.25,1151.00
915.50,1152.25
916.75,1153.50
917.00,1154.75
918.50,1156.00
919.25,1157.25
920.50,1158.50
921.75,1159.75
922.00,1161.00
923.50,1162.25
924.25,1163.50
925.50,1164.75
926.75,1166.00
927.00,1167.25
928.50,1168.50
929.75,1169.75
930.25,1171.00
931.50,1172.25
932.75,1173.50
933.00,1174.75
934.50,1176.00
935.25,1177.25
936.50,1178.50
937.75,1179.75
938.00,1181.00
939.50,1182.25
940.25,1183.50
941.50,1184.75
942.75,1186.00
943.00,1187.25
944.50,1188.50
945.75,1189.75
946.25,1191.00
947.50,1192.25
948.75,1193.50
949.00,1194.75
950.50,1196.00
951.25,1197.25
952.50,1198.50
953.75,1199.75
954.00,1201.00
955.50,1202.25
956.25,1203.50
957.50,1204.75
958.75,1206.00
959.00,1207.25
960.50,1208.50
961.75,1209.75
962.25,1211.00
963.50,1212.25
964.75,1213.50
965.00,1214.75
966.50,1216.00
967.25,1217.25
968.50,1218.50
969.75,1219.75
970.00,1221.00
971.50,1222.25
972.25,1223.50
973.50,1224.75
974.75,1226.00
975.00,1227.25
976.50,1228.50
977.75,1229.75
978.25,1231.00
979.50,1232.25
980.75,1233.50
981.00,1234.75
982.50,1236.00
983.25,1237.25
984.50,1238.50
985.75,1239.75
986.00,1241.00
987.50,1242.25
988.25,1243.50
989.50,1244.75
990.75,1246.00
991.00,1247.25
992.50,1248.50
993.75,1249.75
994.25,1251.00
995.50,1252.25
996.75,1253.50
997.00,1254.75
998.50,1256.00
999.25,1257.25
1000.50,1258.50
1001.75,1259.75
1002.00,1261.00
1003.50,1262.25
1004.25,1263.50
1005.50,1264.75
1006.75,1266.00
1007.00,1267.25
1008.50,1268.50
1009.75,1269.75
1010.25,1271.00
1011.50,1272.25
1012.75,1273.50
1013.00,1274.75
1014.50,1276.00
1015.25,1277.25
1016.50,1278.50
1017.75,1279.75
1018.00,1281.00
1019.50,1282.25
1020.25,1283.50
1021.50,1284.75
1022.75,1286.00
1023.00,1287.25
1024.50,1288.50
1025.75,1289.75
1026.25,1291.00
1027.50,1292.25
1028.75,1293.50
1029.00,1294.75
1030.50,1296.00
1031.25,1297.25
1032.50,1298.50
1033.75,1299.75
1034.00,1301.00
1035.50,1302.25
1036.25,1303.50
1037.50,1304.75
1038.75,1306.00
1039.00,1307.25
1040.50,1308.50
1041.75,1309.75
1042.25,1311.00
1043.50,1312.25
1044.75,1313.50
1045.00,1314.75
1046.50,1316.00
1047.25,1317.25
1048.50,1318.50
1049.75,1319.75
1050.00,1321.00
1051.50,1322.25
1052.25,1323.50
1053.50,1324.75
1054.75,1326.00
1055.00,1327.25
1056.50,1328.50
1057.75,1329.75
1058.25,1331.00
1059.50,1332.25
1060.75,1333.50
1061.00,1334.75
1062.50,1336.00
1063.25,1337.25
1064.50,1338.50
1065.75,1339.75
1066.00,1341.00
1067.50,1342.25
1068.25,1343.50
1069.50,1344.75
1070.75,1346.00
1071.00,1347.25
1072.50,1348.50
1073.75,1349.75
1074.25,1351.00
1075.50,1352.25
1076.75,1353.50
1077.00,1354.75
1078.50,1356.00
1079.25,1357.25
1080.50,1358.50
1081.75,1359.75
1082.00,1361.00
1083.50,1362.25
1084.25,1363.50
1085.50,1364.75
1086.75,1366.00
1087.00,1367.25
1088.50,1368.50
1089.75,1369.75
1090.25,1371.00
1091.50,1372.25
1092.75,1373.50
1093.00,1374.75
1094.50,1376.00
1095.25,1377.25
1096.50,1378.50
1097.75,1379.75
1098.00,1381.00
1099.50,1382.25
1100.25,1383.50
1101.50,1384.75
1102.75,1386.00
1103.00,1387.25
1104.50,1388.50
1105.75,1389.75
1106.25,1391.00
1107.50,1392.25
1108.75,1393.50
1109.00,1394.75
1110.50,1396.00
1111.25,1397.25
1112.50,1398.50
1113.75,1399.75
1114.00,1401.00
1115.50,1402.25
1116.25,1403.50
1117.50,1404.75
1118.75,1406.00
1119.00,1407.25
1120.50,1408.50
1121.75,1409.75
1122.25,1411.00
1123.50,1412.25
1124.75,1413.50
1125.00,1414.75
1126.50,1416.00
1127.25,1417.25
1128.50,1418.50
1129.75,1419.75
1130.00,1421.00
1131.50,1422.25
1132.25,1423.50
1133.50,1424.75
1134.75,1426.00
1135.00,1427.25
1136.50,1428.50
1137.75,1429.75
1138.25,1431.00
1139.50,1432.25
1140.75,1433.50
1141.00,1434.75
1142.50,1436.00
1143.25,1437.25
1144.50,1438.50
1145.75,1439.75
1146.00,1441.00
1147.50,1442.25
1148.25,1443.50
1149.50,1444.75
1150.75,1446.00
1151.00,1447.25
1152.50,1448.50
1153.75,1449.75
1154.25,1451.00
1155.50,1452.25
1156.75,1453.50
1157.00,1454.75
1158.50,1456.00
1159.25,1457.25
1160.50,1458.50
1161.75,1459.75
1162.00,1461.00
1163.50,1462.25
1164.25,1463.50
1165.50,1464.75
1166.75,1466.00
1167.00,1467.25
1168.50,1468.50
1169.75,1469.75
1170.25,1471.00
1171.50,1472.25
1172.75,1473.50
1173.00,1474.75
1174.50,1476.00
1175.25,1477.25
1176.50,1478.50
1177.75,1479.75
1178.00,1481.00
1179.50,1482.25
1180.25,1483.50
1181.50,1484.75
1182.75,1486.00
1183.00,1487.25
1184.50,1488.50
1185.75,1489.75
1186.25,1491.00
1187.50,1492.25
1188.75,1493.50
1189.00,1494.75
1190.50,1496.00
1191.25,1497.25
1192.50,1498.50
1193.75,1499.75
1194.00,1501.00
1195.50,1502.25
1196.25,1503.50
1197.50,1504.75
1198.75,1506.00
1199.00,1507.25
1200.50,1508.50
1201.75,1509.75
1202.25,1511.00
1203.50,1512.25
1204.75,1513.50
1205.00,1514.75
1206.50,1516.00
1207.25,1517.25
1208.50,1518.50
1209.75,1519.75
1210.00,1521.00
1211.50,1522.25
1212.25,1523.50
1213.50,1524.75
1214.75,1526.00
1215.00,1527.25
1216.50,1528.50
1217.75,1529.75
1218.25,1531.00
1219.50,1532.25
1220.75,1533.50
1221.00,1534.75
1222.50,1536.00
1223.25,1537.25
1224.50,1538.50
1225.75,1539.75
1226.00,1541.00
1227.50,1542.25
1228.25,1543.50
1229.50,1544.75
1230.75,1546.00
1231.00,1547.25
1232.50,1548.50
1233.75,1549.75
1234.25,1551.00
1235.50,1552.25
1236.75,1553.50
1237.00,1554.75
1238.50,1556.00
1239.25,1557.25
1240.50,1558.50
1241.75,1559.75
1242.00,1561.00
1243.50,1562.25
1244.25,1563.50
1245.50,1564.75
1246.75,1566.00
1247.00,1567.25
1248.50,1568.50
1249.75,1569.75
1250.25,1571.00
1251.50,1572.25
1252.75,1573.50
1253.00,1574.75
1254.50,1576.00
1255.25,1577.25
1256.50,1578.50
1257.75,1579.75
1258.00,1581.00
1259.50,1582.25
1260.25,1583.50
1261.50,1584.75
1262.75,1586.00
1263.00,1587.25
1264.50,1588.50
1265.75,1589.75
1266.25,1591.00
1267.50,1592.25
1268.75,1593.50
1269.00,1594.75
1270.50,1596.00
1271.25,1597.25
1272.50,1598.50
1273.75,1599.75
1274.00,1601.00
1275.50,1602.25
1276.25,1603.50
1277.50,1604.75
1278.75,1606.00
1279.00,1607.25
1280.50,1608.50
1281.75,1609.75
1282.25,1611.00
1283.50,1612.25
1284.75,1613.50
1285.00,1614.75
1286.50,1616.00
1287.25,1617.25
1288.50,1618.50
1289.75,1619.75
1290.00,1621.00
1291.50,1622.25
1292.25,1623.50
1293.50,1624.75
1294.75,1626.00
1295.00,1627.25
1296.50,1628.50
1297.75,1629.75
1298.25,1631.00
1299.50,1632.25
1300.75,1633.50
1301.00,1634.75
1302.50,1636.00
1303.25,1637.25
1304.50,1638.50
1305.75,1639.75
1306.00,1641.00
1307.50,1642.25
1308.25,1643.50
1309.50,1644.75
1310.75,1646.00
1311.00,1647.25
1312.50,1648.50
1313.75,1649.75
1314.25,1651.00
1315.50,1652.25
1316.75,1653.50
1317.00,1654.75
1318.50,1656.00
1319.25,1657.25
1320.50,1658.50
1321.75,1659.75
1322.00,1661.00
1323.50,1662.25
1324.25,1663.50
1325.50,1664.75
1326.75,1666.00
1327.00,1667.25
1328.50,1668.50
1329.75,1669.75
1330.25,1671.00
1331.50,1672.25
1332.75,1673.50
1333.00,1674.75
1334.50,1676.00
1335.25,1677.25
1336.50,1678.50
1337.75,1679.75
1338.00,1681.00
1339.50,1682.25
1340.25,1683.50
1341.50,1684.75
1342.75,1686.00
1343.00,1687.25
1344.50,1688.50
1345.75,1689.75
1346.25,1691.00
1347.50,1692.25
1348.75,1693.50
1349.00,1694.75
1350.50,1696.00
1351.25,1697.25
1352.50,1698.50
1353.75,1699.75
1354.00,1701.00
1355.50,1702.25
1356.25,1703.50
1357.50,1704.75
1358.75,1706.00
1359.00,1707.25
1360.50,1708.50
1361.75,1709.75
1362.25,1711.00
1363.50,1712.25
1364.75,1713.50
1365.00,1714.75
1366.50,1716.00
1367.25,1717.25
1368.50,1718.50
1369.75,1719.75
1370.00,1721.00
1371.50,1722.25
1372.25,1723.50
1373.50,1724.75
1374.75,1726.00
1375.00,1727.25
1376.50,1728.50
1377.75,1729.75
1378.25,1731.00
1379.50,1732.25
1380.75,1733.50
1381.00,1734.75
1382.50,1736.00
1383.25,1737.25
1384.50,1738.50
1385.75,1739.75
1386.00,1741.00
1387.50,1742.25
1388.25,1743.50
1389.50,1744.75
1390.75,1746.00
1391.00,1747.25
1392.50,1748.50
1393.75,1749.75
1394.25,1751.00
1395.50,1752.25
1396.75,1753.50
1397.00,1754.75
1398.50,1756.00
1399.25,1757.25
1400.50,1758.50
1401.75,1759.75
1402.00,1761.00
1403.50,1762.25
1404.25,1763.50
1405.50,1764.75
1406.75,1766.00
1407.00,1767.25
1408.50,1768.50
1409.75,1769.75
1410.25,1771.00
1411.50,1772.25
1412.75,1773.50
1413.00,1774.75
1414.50,1776.00
1415.25,1777.25
1416.50,1778.50
1417.75,1779.75
1418.00,1781.00
1419.50,1782.25
1420.25,1783.50
1421.50,1784.75
1422.75,1786.00
1423.00,1787.25
1424.50,1788.50
1425.75,1789.75
1426.25,1791.00
1427.50,1792.25
1428.75,1793.50
1429.00,1794.75
1430.50,1796.00
1431.25,1797.25
1432.50,1798.50
1433.75,1799.75
1434.00,1801.00
1435.50,1802.25
1436.25,1803.50
1437.50,1804.75
1438.75,1806.00
1439.00,1807.25
1440.50,1808.50
1441.75,1809.75
1442.25,1811.00
1443.50,1812.25
1444.75,1813.50
1445.00,1814.75
1446.50,1816.00
1447.25,1817.25
1448.50,1818.50
1449.75,1819.75
1450.00,1821.00
1451.50,1822.25
1452.25,1823.50
1453.50,1824.75
1454.75,1826.00
1455.00,1827.25
1456.50,1828.50
1457.75,1829.75
1458.25,1831.00
1459.50,1832.25
1460.75,1833.50
1461.00,1834.75
1462.50,1836.00
1463.25,1837.25
1464.50,1838.50
1465.75,1839.75
1466.00,1841.00
1467.50,1842.25
1468.25,1843.50
1469.50,1844.75
1470.75,1846.00
1471.00,1847.25
1472.50,1848.50
1473.75,1849.75
1474.25,1851.00
1475.50,1852.25
1476.75,1853.50
1477.00,1854.75
1478.50,1856.00
1479.25,1857.25
1480.50,1858.50
1481.75,1859.75
1482.00,1861.00
1483.50,1862.25
1484.25,1863.50
1485.50,1864.75
1486.75,1866.00
1487.00,1867.25
1488.50,1868.50
1489.75,1869.75
1490.25,1871.00
1491.50,1872.25
1492.75,1873.50
1493.00,1874.75
1494.50,1876.00
1495.25,1877.25
1496.50,1878.50
1497.75,1879.75
1498.00,1881.00
1499.50,1882.25
1500.25,1883.50
1501.50,1884.75
1502.75,1886.00
1503.00,1887.25
1504.50,1888.50
1505.75,1889.75
1506.25,1891.00
1507.50,1892.25
1508.75,1893.50
1509.00,1894.75
1510.50,1896.00
1511.25,1897.25
1512.50,1898.50
1513.75,1899.75
1514.00,1901.00
1515.50,1902.25
1516.25,1903.50
1517.50,1904.75
1518.75,1906.00
1519.00,1907.25
1520.50,1908.50
1521.75,1909.75
1522.25,1911.00
1523.50,1912.25
1524.75,1913.50
1525.00,1914.75
1526.50,1916.00
1527.25,1917.25
1528.50,1918.50
1529.75,1919.75
1530.00,1921.00
1531.50,1922.25
1532.25,1923.50
1533.50,1924.75
1534.75,1926.00
1535.00,1927.25
1536.50,1928.50
1537.75,1929.75
1538.25,1931.00
1539.50,1932.25
1540.75,1933.50
1541.00,1934.75
1542.50,1936.00
1543.25,1937.25
1544.50,1938.50
1545.75,1939.75
1546.00,1941.00
1547.50,1942.25
1548.25,1943.50
1549.50,1944.75
1550.75,1946.00
1551.00,1947.25
1552.50,1948.50
1553.75,1949.75
1554.25,1951.00
1555.50,1952.25
1556.75,1953.50
1557.00,1954.75
1558.50,1956.00
1559.25,1957.25
1560.50,1958.50
1561.75,1959.75
1562.00,1961.00
1563.50,1962.25
1564.25,1963.50
1565.50,1964.75
1566.75,1966.00
1567.00,1967.25
1568.50,1968.50
1569.75,1969.75
1570.25,1971.00
1571.50,1972.25
1572.75,1973.50
1573.00,1974.75
1574.50,1976.00
1575.25,1977.25
1576.50,1978.50
1577.75,1979.75
1578.00,1981.00
1579.50,1982.25
1580.25,1983.50
1581.50,1984.75
1582.75,1986.00
1583.00,1987.25
1584.50,1988.50
1585.75,1989.75
1586.25,1991.00
1587.50,1992.25
1588.75,1993.50
1589.00,1994.75
1590.50,1996.00
1591.25,1997.25
1592.50,1998.50
1593.75,1999.75
1594.00,2001.00
1595.50,2002.25
1596.25,2003.50
1597.50,2004.75
1598.75,2006.00
1599.00,2007.25
1600.50,2008.50
1601.75,2009.75
1602.25,2011.00
1603.50,2012.25
1604.75,2013.50
1605.00,2014.75
1606.50,2016.00
1607.25,2017.25
1608.50,2018.50
1609.75,2019.75
1610.00,2021.00
1611.50,2022.25
1612.25,2023.50
1613.50,2024.75
1614.75,2026.00
1615.00,2027.25
1616.50,2028.50
1617.75,2029.75
1618.25,2031.00
1619.50,2032.25
1620.75,2033.50
1621.00,2034.75
1622.50,2036.00
1623.25,2037.25
1624.50,2038.50
1625.75,2039.75
1626.00,2041.00
1627.50,2042.25
1628.25,2043.50
1629.50,2044.75
1630.75,2046.00
1631.00,2047.25
1632.50,2048.50
1633.75,2049.75
1634.25,2051.00
1635.50,2052.25
1636.75,2053.50
1637.00,2054.75
1638.50,2056.00
1639.25,2057.25
1640.50,2058.50
1641.75,2059.75
1642.00,2061.00
1643.50,2062.25
1644.25,2063.50
1645.50,2064.75
1646.75,2066.00
1647.00,2067.25
1648.50,2068.50
1649.75,2069.75
1650.25,2071.00
1651.50,2072.25
1652.75,2073.50
1653.00,2074.75
1654.50,2076.00
1655.25,2077.25
1656.50,2078.50
1657.75,2079.75
1658.00,2081.00
1659.50,2082.25
1660.25,2083.50
1661.50,2084.75
1662.75,2086.00
1663.00,2087.25
1664.50,2088.50
1665.75,2089.75
1666.25,2091.00
1667.50,2092.25
1668.75,2093.50
1669.00,2094.75
1670.50,2096.00
1671.25,2097.25
1672.50,2098.50
1673.75,2099.75
1674.00,2101.00
1675.50,2102.25
1676.25,2103.50
1677.50,2104.75
1678.75,2106.00
1679.00,2107.25
1680.50,2108.50
1681.75,2109.75
1682.25,2111.00
1683.50,2112.25
1684.75,2113.50
1685.00,2114.75
1686.50,2116.00
1687.25,2117.25
1688.50,2118.50
1689.75,2119.75
1690.00,2121.00
1691.50,2122.25
1692.25,2123.50
1693.50,2124.75
1694.75,2126.00
1695.00,2127.25
1696.50,2128.50
1697.75,2129.75
1698.25,2131.00
1699.50,2132.25
1700.75,2133.50
1701.00,2134.75
1702.50,2136.00
1703.25,2137.25
1706.50,2138.50
1705.75,2139.75
1706.00,2141.00
1707.50,2142.25
1708.25,2143.50
1709.50,2144.75
1710.75,2146.00
1711.00,2147.25
1712.50,2148.50
1713.75,2149.75
1714.25,2151.00
1715.50,2152.25
1716.75,2153.50
1717.00,2154.75
1718.50,2156.00
1719.25,2157.25
1720.50,2158.50
1721.75,2159.75
1722.00,2161.00
1723.50,2162.25
1724.25,2163.50
1725.50,2164.75
1726.75,2166.00
1727.00,2167.25
1728.50,2168.50
1729.75,2169.75
1730.25,2171.00
1731.50,2172.25
1732.75,2173.50
1733.00,2174.75
1734.50,2176.00
1735.25,2177.25
1736.50,2178.50
1737.75,2179.75
1738.00,2181.00
1739.50,2182.25
1740.25,2183.50
1741.50,2184.75
1742.75,2186.00
1743.00,2187.25
1744.50,2188.50
1745.75,2189.75
1746.25,2191.00
1747.50,2192.25
1748.75,2193.50
1749.00,2194.75
1750.50,2196.00
1751.25,2197.25
1752.50,2198.50
1753.75,2199.75
1754.00,2201.00
1755.50,2202.25
1756.25,2203.50
1757.50,2204.75
1758.75,2206.00
1759.00,2207.25
1760.50,2208.50
1761.75,2209.75
1762.25,2211.00
1763.50,2212.25
1764.75,2213.50
1765.00,2214.75
1766.50,2216.00
1767.25,2217.25
1768.50,2218.50
1769.75,2219.75
1770.00,2221.00
1771.50,2222.25
1772.25,2223.50
1773.50,2224.75
1774.75,2226.00
1775.00,2227.25
1776.50,2228.50
1777.75,2229.75
1778.25,2231.00
1779.50,2232.25
1780.75,2233.50
1781.00,2234.75
1782.50,2236.00
1783.25,2237.25
1784.50,2238.50
1785.75,2239.75
1786.00,2241.00
1787.50,2242.25
1788.25,2243.50
1789.50,2244.75
1790.75,2246.00
1791.00,2247.25
1792.50,2248.50
1793.75,2249.75
1794.25,2251.00
1795.50,2252.25
1796.75,2253.50
1797.00,2254.75
1798.50,2256.00
1799.25,2257.25
1800.50,2258.50
1801.75,2259.75
1802.00,2261.00
1803.50,2262.25
1804.25,2263.50
1805.50,2264.75
1806.75,2266.00
1807.00,2267.25
1808.50,2268.50
1809.75,2269.75
1810.25,2271.00
1811.50,2272.25
1812.75,2273.50
1813.00,2274.75
1814.50,2276.00
1815.25,2277.25
1816.50,2278.50
1817.75,2279.75
1818.00,2281.00
1819.50,2282.25
1820.25,2283.50
1821.50,2284.75
1822.75,2286.00
1823.00,2287.25
1824.50,2288.50
1825.75,2289.75
1826.25,2291.00
1827.50,2292.25
1828.75,2293.50
1829.00,2294.75
1830.50,2296.00
1831.25,2297.25
1832.50,2298.50
1833.75,2299.75
1834.00,2301.00
1835.50,2302.25
1836.25,2303.50
1837.50,2304.75
1838.75,2306.00
1839.00,2307.25
1840.50,2308.50
1841.75,2309.75
1842.25,2311.00
1843.50,2312.25
1844.75,2313.50
1845.00,2314.75
1846.50,2316.00
1847.25,2317.25
1848.50,2318.50
1849.75,2319.75
1850.00,2321.00
1851.50,2322.25
1852.25,2323.50
1853.50,2324.75
1854.75,2326.00
1855.00,2327.25
1856.50,2328.50
1857.75,2329.75
1858.25,2331.00
1859.50,2332.25
1860.75,2333.50
1861.00,2334.75
1862.50,2336.00
1863.25,2337.25
1864.50,2338.50
1865.75,2339.75
1866.00,2341.00
1867.50,2342.25
1868.25,2343.50
1869.50,2344.75
1870.75,2346.00
1871.00,2347.25
1872.50,2348.50
1873.75,2349.75
1874.25,2351.00
1875.50,2352.25
1876.75,2353.50
1877.00,2354.75
1878.50,2356.00
1879.25,2357.25
1880.50,2358.50
1881.75,2359.75
1882.00,2361.00
1883.50,2362.25
1884.25,2363.50
1885.50,2364.75
1886.75,2366.00
1887.00,2367.25
1888.50,2368.50
1889.75,2369.75
1890.25,2371.00
1891.50,2372.25
1892.75,2373.50
1893.00,2374.75
1894.50,2376.00
1895.25,2377.25
1896.50,2378.50
1897.75,2379.75
1898.00,2381.00
1899.50,2382.25
1900.25,2383.50
1901.50,2384.75
1902.75,2386.00
1903.00,2387.25
1904.50,2388.50
1905.75,2389.75
1906.25,2391.00
1907.50,2392.25
1908.75,2393.50

Writing stocks2.txt


In [53]:
%%writefile atv11.cu
#include <thrust/device_vector.h>   // Thrust library for GPU vectors
#include <thrust/transform.h>
#include <thrust/host_vector.h>     // Thrust library for CPU vectors
#include <iostream>
#include <chrono>                   // Library for timing code execution
#include <sstream>

int main() {
    int n = 3520;  // Define the size of the vector
    std::chrono::duration<double> diff; // Variable to store the time difference

    thrust::host_vector<double> AAPL(n, 0);
    thrust::host_vector<double> MSFT(n, 0);

    auto leitura_i = std::chrono::steady_clock::now();

    std::string input;
    int i = 0;
    while (i < n && std::getline(std::cin, input)) {
        std::stringstream ss(input);
        std::string str_value1, str_value2;

        // Separate values before and after the comma
        if (std::getline(ss, str_value1, ',') && std::getline(ss, str_value2, ',')) {
            AAPL[i] = std::stod(str_value1);
            MSFT[i] = std::stod(str_value2);
            i++;
        }
    }

    auto leitura_f = std::chrono::steady_clock::now(); // End of reading time measurement
    diff = leitura_f - leitura_i;

    std::cout << "Tempo de LEITURA (em segundos): " << diff.count() << "\n" << std::endl;

    auto copia_i = std::chrono::steady_clock::now();

    thrust::device_vector<double> dev1 = AAPL;
    thrust::device_vector<double> dev2 = MSFT;

    auto copia_f = std::chrono::steady_clock::now();
    diff = copia_f - copia_i;

    std::cout << "Tempo de CÓPIA (em segundos): " << diff.count() << "\n" << std::endl;

    auto transform_i = std::chrono::steady_clock::now();

    thrust::device_vector<double> diff_transf(n);

    thrust::transform(dev1.begin(), dev1.end(), dev2.begin(), diff_transf.begin(), thrust::minus<double>());

    auto transform_f = std::chrono::steady_clock::now();
    diff = transform_f - transform_i;

    std::cout << "Tempo de TRANSFORMACAO (em segundos): " << diff.count() << "\n" << std::endl;

    auto mean_i = std::chrono::steady_clock::now();

    double soma = thrust::reduce(diff_transf.begin(), diff_transf.end(), 0, thrust::plus<double>());

    double mean = soma / n;


    auto mean_f = std::chrono::steady_clock::now();
    diff = mean_f - mean_i;

    std::cout << "Media: " << mean  << std::endl;
    std::cout << "Tempo de Media das diferenças (em segundos): " << diff.count() << "\n" << std::endl;

    return 0;
}


Overwriting atv11.cu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [54]:
!nvcc -arch=sm_70 -std=c++14 atv11.cu -o atv11

In [55]:
!./atv11 < stocks2.txt

Tempo de LEITURA (em segundos): 0.00416015

Tempo de CÓPIA (em segundos): 0.10394

Tempo de TRANSFORMACAO (em segundos): 0.00052834

Media: -265.232
Tempo de Media das diferenças (em segundos): 0.000134276

